In [231]:
pip install -q langchain-chroma langchain_community langchain_text_splitters langchain_groq langgraph pypdf PyPDF2 langchain_huggingface faiss-cpu langchain_google_genai

In [232]:
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.prompts import PromptTemplate
from langchain_community.llms import HuggingFaceEndpoint
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_community.vectorstores.faiss import FAISS
from langchain.docstore.document import Document

In [1]:
from dotenv import load_dotenv
load_dotenv()

import os
print(os.environ['LANGCHAIN_API_KEY'])


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


In [234]:
from langchain_google_genai import GoogleGenerativeAI
llm = GoogleGenerativeAI(model="gemini-1.0-pro-latest", google_api_key=userdata.get('GOOGLE_API_KEY'),temperature=0.2)


In [235]:
# import os

# os.environ["GROQ_API_KEY"] = "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"

# from langchain_groq import ChatGroq

# llm = ChatGroq(model="llama3-8b-8192",temperature=0)

In [236]:
chara_text_splitter = CharacterTextSplitter()
recur_text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200
)


In [263]:
chara_text_splitter = CharacterTextSplitter()
recur_text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200
)


def preprocessing_text(data):
    texts = chara_text_splitter.split_text(data)
    doc_data = [Document(page_content=t) for t in texts]
    text_splits = recur_text_splitter.split_documents(doc_data)
    return text_splits


def create_retriever(text_splits):
    embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
    vector_db = FAISS.from_documents(text_splits, embedding_model)
    retriever = vector_db.as_retriever(search_type="mmr", search_kwargs={"k": 5})
    return retriever

In [264]:
pdf_reader = PdfReader("/content/Tender Information for document filling.pdf")
text = ""
for page in pdf_reader.pages:
    text += page.extract_text()

preprocessed_text = preprocessing_text(text)
retriever = create_retriever(preprocessed_text)


############
# from langchain.document_loaders import PyPDFLoader
# file_path = "/content/Tender Information for document filling.pdf"
# loader = PyPDFLoader(file_path)
# documents = loader.load()
# retriever = create_retriever(documents)

In [239]:
from langchain_google_genai import GoogleGenerativeAI
llm = GoogleGenerativeAI(model="gemini-1.0-pro-latest", google_api_key=userdata.get('GOOGLE_API_KEY'),temperature=0.2)


In [278]:
from langchain_core.prompts import ChatPromptTemplate
# from langchain_core.prompts import PromptTemplate

# template = """
# you are specialized AI assistant that fill the form based on information provided
# to you.The information is regarding a tender information below:
# {context}

# Given input is of type form where fields need to be filled
# {input}

# the goal is to create a filled version of the input and return in the same format

# * just output the filled information in the input format

# example input/output

# input:
# Company Name: __________________________________________2. Company Address: ________________________________________

# output:
# Company Name:  ford ompany 2. Company Address: New york USA
# """

from langchain.prompts import PromptTemplate
template1 = """
Fill in the blanks of the following form based on the provided context.

**Context:**
{context}

**Form:**
{input}

* make sure the format of the form kept unchanged like headings, numberings
* Form should be complete in length in the output as provided
"""
# template = """
# You are required to fill in the blanks in the form below by only using the provided context.

# **Context:**
# {context}

# **Form:**
# {input}

# ***** if no fields(____) found in the **Form** output will be the provided **Form** only ****

# Instructions:
# 1. Complete **all** sections of the form based on the context provided.
# 2. Maintain the original structure, headings, and numbering exactly as they are presented in the form.
# """
prompt1 = PromptTemplate.from_template(template1)
template2 = """
Fill in the blanks of the following form based on the provided context.

**Context:**
{context}

**Form:**
{input}

***** if no fields(____) found in the **Form** output will be the provided **Form** only ****
"""
prompt2 = PromptTemplate.from_template(template2)
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


# prompt = select_prompt(input)
rag_chain = (
  {"context": retriever | format_docs, "input": RunnablePassthrough()}
  | prompt1
  | llm
  # | StrOutputParser()
)
print(rag_chain.invoke(docts[1].page_content))

 Handover and commissioning  
4. Tender Packages  
Bidders are invited to submit proposals for one of the following packages:  
4.1 Package A: Full Development  
This package includes the complete scope of work as outlined in Section 3, covering all aspects 
of the project from site preparation to final handover.  
Key Components:  
 Complete construction of all structures  
 Full interior fit -out 
 Comprehensive landscaping  
 Installation of all utilities and systems  
 Project management for the entire development  
Estimated Timeline: 36 months  
4.2 Package B: Core and Shell Development  
This package focuses on the development of the main structure and essential systems, leaving 
interior fit -out and some finishing works for  future completion.  
Key Components:  
 Site preparation and earthwork  
 Construction of the main structure(s)  
 Installation of core building systems (HVAC, electrical, plumbing)  
 Basic exterior finishing  
 Minimal landscaping  
Estimated 

In [243]:

print(docts[3].page_content)

 
By submitting this form, I certify that the information provid ed is true and accurate to the best of 
my knowledge.  
Signature: ___________________________ Date: _______________  
Printed Name: ________________________ Title: _______________  
5. Submission Requirements  
Bidders must submit the following documents:  
1. Company Profile  
o Legal structure and registration details  
o Financial statements for the past 3 years  
o Relevant experience and completed projects  
2. Technical Proposal  
o Detailed methodology and approach  
o Work breakdown structure  
o Project timeline and milestones  
o Quality assurance plan  
o Health and safety plan  
3. Financial Prop osal  
o Detailed cost breakdown  
o Payment schedule  
o Financial assumptions and exclusions  
4. Resource Plan  
o Key personnel and their qualifications  
o Subcontractor information (if applicable)  
o Equipment and machinery to be used  
5. Certifications and Compliance  
o Relevant i ndustry certifications  
o

In [132]:
print(rag_chain.invoke("1. Company Name: __________________________________________2. Company Address: ________________________________________"))

1. Company Name: OptimumTech Development
2. Company Address: Riyadh, Saudi Arabia


In [110]:
from langchain.document_loaders import PyPDFLoader
file_path = "/content/Real Estate Development Tender Document.pdf"
loader = PyPDFLoader(file_path)
docts = loader.load()

In [134]:
print(docts[0].page_content)

Real Estate Development Tender Document  
Table of Contents  
1. Introdu ction  
2. Project Overview  
3. Scope of Work  
4. Tender Packages 4.1 Package A 4.2 Package B        
5. Submission Requirements  
6. Evaluation Criteria  
7. Terms and Conditions  
8. Contact Information  
1. Introduction  
Greenfield Developments LLC is seeking proposals from qualified contractor s for the 
development of a mixed -use complex located at 123 Prosperity Avenue, Metropolis, FL 33101. 
This tender document outlines the project requirements, scope of work, and submission 
guidelines for interested parties.  
2. Project Overview  
Project Name: Metropolis Central Location: 123 Prosperity Avenue, Metropolis, FL 33101 Total 
Area: 50,000 square meters Proposed Development: Mixed -use complex with residential 
apartments, office spaces, and retail outlets  
The project aims to create a sustainable, moder n urban living and working environment that 
enhances the cityscape of Metropolis while providing h

In [35]:
from PyPDF2


In [144]:
print(rag_chain.invoke(docts[2].page_content))

2. Company Address: Riyadh, Saudi Arabia  
3. Primary Contact Person: Name: Ahmed Al -Saud  Title: CEO  Phone: +966 123456789  Email: ahmed.alsaud@optimumtech.sa  
4. Years of Experience in Real Estate Development: 15  
5. Number of Similar Projects Completed: 10  
6. Annual Turnover (Last 3 Years): Year 1: $100 million  Year 2: $120 million  Year 3: $150 million  
7. Proposed Project Timeline: 24  months  
8. Estimated Total Project Cost: $200 million  
9. Key Subcontractors (if any): a. 
______________________________________________________ b. 
______________________________________________________ c. 
______________________________________________________  
10. Brief Project Approach (max 500 words):  
 
 
 
 
 
11. Certifications and Accreditations:  
 
 
12. References (provide at least two): a. Company: 
___________________ _________________________ Contact Person: 
_______________________________________ Phone/Email: 
_________________________________________ b. Company: 
_____

In [269]:
output_text = ""
filledpages = []

def select_prompt(input):
  if '____' in input:
    return prompt1
  else:
    return prompt2

for document in docts:
  prompt = select_prompt(document.page_content)
  rag_chain = (
    {"context": retriever | format_docs, "input": RunnablePassthrough()}
    | prompt
    | llm
    # | StrOutputParser()
  )
  out = rag_chain.invoke(document.page_content)
  # if len(out) < len(document.page_content)/2:
  #   print("output waay less than input Retaining input..")
  #   filledpages.append(Document(page_content=document.page_content ))
  #   output_text +=document.page_content
  # else:
  filledpages.append(Document(page_content= out))
  output_text +=out


In [260]:
print(filledpages[3].page_content)

 
By submitting this form, I certify that the information provid ed is true and accurate to the best of 
my knowledge.  
Signature: ___________________________ Date: _______________  
Printed Name: ________________________ Title: _______________  
5. Submission Requirements  
Bidders must submit the following documents:  
1. Company Profile  
o Legal structure and registration details  
o Financial statements for the past 3 years  
o Relevant experience and completed projects  
2. Technical Proposal  
o Detailed methodology and approach  
o Work breakdown structure  
o Project timeline and milestones  
o Quality assurance plan  
o Health and safety plan  
3. Financial Prop osal  
o Detailed cost breakdown  
o Payment schedule  
o Financial assumptions and exclusions  
4. Resource Plan  
o Key personnel and their qualifications  
o Subcontractor information (if applicable)  
o Equipment and machinery to be used  
5. Certifications and Compliance  
o Relevant i ndustry certifications  
o

In [270]:
print(output_text)

Real Estate Development Tender Document  
Table of Contents  
1. Introdu ction  
2. Project Overview  
3. Scope of Work  
4. Tender Packages 4.1 Package A 4.2 Package B        
5. Submission Requirements  
6. Evaluation Criteria  
7. Terms and Conditions  
8. Contact Information  
1. Introduction  
Greenfield Developments LLC is seeking proposals from qualified contractor s for the 
development of a mixed -use complex located at 123 Prosperity Avenue, Metropolis, FL 33101. 
This tender document outlines the project requirements, scope of work, and submission 
guidelines for interested parties.  
2. Project Overview  
Project Name: Metropolis Central Location: 123 Prosperity Avenue, Metropolis, FL 33101 Total 
Area: 50,000 square meters Proposed Development: Mixed -use complex with residential 
apartments, office spaces, and retail outlets  
The project aims to create a sustainable, moder n urban living and working environment that 
enhances the cityscape of Metropolis while providing h

In [225]:
print(output_text)

Real Estate Development Tender Document  
Table of Contents  
1. Introdu ction  
2. Project Overview  
3. Scope of Work  
4. Tender Packages 4.1 Package A 4.2 Package B        
5. Submission Requirements  
6. Evaluation Criteria  
7. Terms and Conditions  
8. Contact Information  
1. Introduction  
Greenfield Developments LLC is seeking proposals from qualified contractor s for the 
development of a mixed -use complex located at 123 Prosperity Avenue, Metropolis, FL 33101. 
This tender document outlines the project requirements, scope of work, and submission 
guidelines for interested parties.  
2. Project Overview  
Project Name: Metropolis Central Location: 123 Prosperity Avenue, Metropolis, FL 33101 Total 
Area: 50,000 square meters Proposed Development: Mixed -use complex with residential 
apartments, office spaces, and retail outlets  
The project aims to create a sustainable, moder n urban living and working environment that 
enhances the cityscape of Metropolis while providing h

In [26]:
print(documents[2].page_content)

2. Company Address: ___________________________ _____________  
3. Primary Contact Person: Name: 
_________________________________________________ Title: 
_________________________________________________ Phone: 
________________________________________________ Email: 
__________________________________________ ______  
4. Years of Experience in Real Estate Development: _______________  
5. Number of Similar Projects Completed: _______________________  
6. Annual Turnover (Last 3 Years): Year 1: $_____________________ Year 2: 
$_____________________ Year 3: $____________________ _ 
7. Proposed Project Timeline: _________________________________ months  
8. Estimated Total Project Cost: $_______________________________  
9. Key Subcontractors (if any): a. 
______________________________________________________ b. 
______________________________________________________ c. 
______________________________________________________  
10. Brief Project Approach (max 500 words):  
 
 
 
 
 
11

In [276]:
# from PyPDF2 import PdfReader
# reader = PdfReader("/content/Real Estate Development Tender Document.pdf")


In [279]:

# Open a file in write mode ('w') and save the string
with open('output.txt', 'w') as file:
    file.write(output_text)



{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_x48m', 'function': {'arguments': '{"config":{"run_name":"certification_and_accreditation"}}', 'name': 'PDF_info_retriever'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 82, 'prompt_tokens': 1163, 'total_tokens': 1245, 'completion_time': 0.064195545, 'prompt_time': 0.286992673, 'queue_time': None, 'total_time': 0.351188218}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_af05557ca2', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-be31c12a-e771-4ca5-8ffd-6ecd5e9d59ac-0', tool_calls=[{'name': 'PDF_info_retriever', 'args': {'config': {'run_name': 'certification_and_accreditation'}}, 'id': 'call_x48m', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1163, 'output_tokens': 82, 'total_tokens': 1245})]}}
----
{'tools': {'messages': [ToolMessage(content='Error: AttributeError("\'dict\' object has no attribute \'replace\'")\n Please fix your